# 计算图


In [120]:
import torch

a = torch.eye(4, 3) # 生成一个4x3的单位矩阵
b = torch.full_like(a, 1)   # 生成一个与a形状相同的全1矩阵
x = torch.add(a, 1, b)  # x = a + b,1是可选的缩放因子
w = torch.randn(len(x[0]), 1, requires_grad=True)   # 生成一个随机数列，长度为x的列数，1是输出的列数
# ---------------------------------上方是用户创建，下方是计算得到-------------------------------------------------
y = torch.mm(x, w)  # 矩阵相乘
z = 0.5*y**2    # 矩阵平方
m = (z - 1)/2   # 矩阵减1再除2
n = m - 0.1 # 矩阵减0.1

# 用户创建的是叶子节点
叶子节点在经过反向传播之后梯度值能够得以保留，非叶子节点的梯度值则为：None，


In [121]:
print(f'a.is_leaf:{a.is_leaf}')
print(f'b.is_leaf:{b.is_leaf}')
print(f'x.is_leaf:{x.is_leaf}')
print(f'w.is_leaf:{w.is_leaf}')


a.is_leaf:True
b.is_leaf:True
x.is_leaf:True
w.is_leaf:True


In [122]:

print(f'y.is_leaf:{y.is_leaf}')
print(f'z.is_leaf:{z.is_leaf}')
print(f'm.is_leaf:{m.is_leaf}')
print(f'n.is_leaf:{n.is_leaf}')

y.is_leaf:False
z.is_leaf:False
m.is_leaf:False
n.is_leaf:False


In [123]:
print(x.requires_grad)# 它由用户创建， requires_grad 默认False
print(m.requires_grad)# 它计算得到， requires_grad 默认True


False
True


In [124]:
# n.retain_grad()         # 保留n的梯度 不加报错： UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor
print('grad_w:', w.grad, '\ngrad_n:', n.grad)

grad_w: None 
grad_n: None


/tmp/ipykernel_1736/67028844.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/build/aten/src/ATen/core/TensorBody.h:480.)
  print('grad_w:', w.grad, '\ngrad_n:', n.grad)


# 叶子节点辅助运算，但是最后默认都释放了。所以在一次反向传播完成前叶子结点的值不能在原地址上被修改
p-->x-->y



In [125]:
# 创建一个张量，设置 requires_grad=True
p = torch.ones(3, 4, requires_grad=True)
# 对 p 进行操作
x = torch.sin(p)
y=x*2
# 打印 p 的形状
print(p.shape)
# 由于 x 不是标量，需要传递一个与 x 形状相同的梯度
gradient = torch.ones_like(x)
x.retain_grad() #不保留会被释放
# 执行反向传播，计算梯度
y.backward(gradient)
# 查看 p 的梯度
print(x.grad)   # y=2x,dy/dx=2,所以x.grad=2
print(p.grad)   # y=2sin(p),dy/dp=2cos(p),所以p.grad=2cos(p),cos1=0.5403


torch.Size([3, 4])
tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]])
tensor([[1.0806, 1.0806, 1.0806, 1.0806],
        [1.0806, 1.0806, 1.0806, 1.0806],
        [1.0806, 1.0806, 1.0806, 1.0806]])
